In [167]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [168]:
DATA = "esquistossomose.parquet"
codigos_estados = {35: 'SP', 41: 'PR', 42: 'SC', 43: 'RS', 50: 'MS', 11: 'RO', 12: 'AC', 13: 'AM', 14: 'RR', 15: 'PA', 16: 'AP', 17: 'TO', 21: 'MA', 24: 'RN', 25: 'PB', 26: 'PE', 27: 'AL', 28: 'SE', 29: 'BA', 31: 'MG', 33: 'RJ', 51: 'MT', 52: 'GO', 53: 'DF', 22: 'PI', 23: 'CE', 32: 'ES'}

# Questão 1

Para essa questão, foi notado que o comprimento do index seria igual o número de casos da questão.    
Assim, basta ler o arquivo e calcular o comprimento do index da variável.

In [169]:
def questao_1(datapath = DATA):
    tabela = pd.read_parquet(datapath)
    # Procura o comprimento do index, que é a quantidade de casos notificados
    tamanho = len(tabela.index)
    return tamanho
questao_1()

4219

# Questão 3
Para essa questão, foi utilizada a função `value_counts` presente na biblioteca do Pandas, a qual serve para agrupar todos os dados presentes numa tabela, contando a quantidade de valores desses dados.  

A função cria um objeto que é uma "series", o qual pode facilmente ser convertido em um dicionário, com a função `dict`.  
Após isso, foi feita uma lista com as chaves do dicionário que foi criado.  

Como a função `value_counts` sempre ordena os valores do maior para o menor, sabemos que quando o primeiro elemento for pego, teremos o sexo com maior quantidade de aparições.  

Por fim, basta retornar tanto a string com o sexo que teve a maior quantidade de aparições quanto o dicionário com os sexos e suas quantidades.

In [170]:
def questao_3(datapath = DATA):
    tabela = pd.read_parquet(datapath)
    # Criando um dicionário com os sexos e a quantidade de ocorrências de homem e mulher
    casos_por_sexo = dict(tabela["CS_SEXO"].value_counts())
    # Criando lista com as quantidades de casos
    lista_com_casos = list(casos_por_sexo.keys())
    # Pegando o maior elemento (primeiro elemento)
    maior_quantidade = lista_com_casos[0]
    # Retornando o sexo que aparece em maior quantidade, bem como a frequência
    return maior_quantidade, casos_por_sexo
questao_3()

('M', {'M': 2558, 'F': 1661})

# Questão 7
Para essa questão, além da variável "codigos_estados" já definida, foi definido um dicionário no qual as chaves são os códigos dos estados e os valores, a quantidade de municípios dos estados.  

Utilizamos um `groupby` nos estados, contando a quantidade de municípios que apareciam. Para evitar a repetição de termos, utilizamos a função `nunique`, a qual retira os municípios que seriam contados duas vezes.  

Após isso, criaremos um dataframe auxiliar no qual tranformaremos o dicionário que temos num dataframe e adicionaremos uma nova coluna a ele com a quantidade de casos por estado.  

Agora, temos um dataframe com a quantidade de cidades de cada UF, bem como a quantidade de cidades que apresentaram a doença.  Portanto, podemos criar uma nova coluna, como sendo a divisão de uma pela outra, para termos a proporção e multiplicaremos por 100 para a obtenção da porcentagem.   

Para obtermos as siglas das UFs, utilizaremos a função `map`, para assim auxiliar na conversão dos códigos nas siglas das unidades da federação.  

Por fim, vamos criar um dicionário no qual vamos zipar a coluna UF com seu respectivo valor de porcentagem e retorná-lo.  

Assim temos uma função que nos retorna cada UF com a respectiva proporção de municípios que relataram casos em cada estado.

In [171]:
def questao_7(datapath = DATA):
    tabela = pd.read_parquet(datapath)
    # Dicionário com o código do estado e a quantidade de municípios nele
    mun_por_est = {31 : 853, 35 : 645, 43 : 497, 29 : 417, 41 : 399, 42 : 295, 52 : 246, 22 : 224, 25 : 223, 21: 217, 26 : 184, 23 : 184, 24 : 167, 15 : 144, 51 : 141, 17 : 139, 27 : 102, 33 : 92, 50 : 79, 32 : 78, 28 : 75, 13 : 62,11 : 52, 12 : 22, 16 : 16, 14 : 15}
    # Agrupando os dados por estado e contando cada município uma única vez
    casos_por_estado = tabela.groupby('SG_UF_NOT')['ID_MUNICIP'].nunique()
    # Convertendo o index para um número para não dar problema no dicionário
    casos_por_estado.index = pd.to_numeric(casos_por_estado.index)
    # Criando dataframe auxiliar
    tabela_auxiliar = pd.DataFrame({"COD": mun_por_est.keys(), "Quantidade de Cidades": mun_por_est.values()})
    # Colocando a quantidade de cidades com casos como uma coluna na tabela
    tabela_auxiliar["Quantidade de cidades com casos"] = tabela_auxiliar["COD"].map(casos_por_estado).fillna(0)
    # Criando a coluna "Porcentagem"
    tabela_auxiliar["Porcentagem"] = (tabela_auxiliar["Quantidade de cidades com casos"]*100/tabela_auxiliar["Quantidade de Cidades"]).round(2)
    # Criando uma coluna com as siglas de cada UF
    tabela_auxiliar["UF"] = tabela_auxiliar["COD"].map(codigos_estados)
    # Criando um dicionário com as siglas e as porcentagens
    dicionario = dict(zip(tabela_auxiliar["UF"], tabela_auxiliar["Porcentagem"]))
    # Retornando o dicionário
    return dicionario
questao_7()

{'MG': 17.82,
 'SP': 16.43,
 'RS': 0.4,
 'BA': 22.54,
 'PR': 4.76,
 'SC': 3.39,
 'GO': 3.66,
 'PI': 0.45,
 'PB': 5.83,
 'MA': 3.69,
 'PE': 17.39,
 'CE': 5.98,
 'RN': 2.4,
 'PA': 3.47,
 'MT': 3.55,
 'TO': 2.16,
 'AL': 30.39,
 'RJ': 16.3,
 'MS': 6.33,
 'ES': 42.31,
 'SE': 32.0,
 'AM': 0.0,
 'RO': 21.15,
 'AC': 0.0,
 'AP': 12.5,
 'RR': 13.33}

# Questão 9
Nessa questão utilizaremos as colunas já previamente realizadas na questão 8. Assim, apenas copiaremos o código.  

No entanto, como o formato presente na coluna não é favorável à manipulação dos dados, transformaremos as datas do tipo "deltatime" para inteiros com o método `dt.days`.  

Após isso, foi criado um dataframe com os dados agrupados por UF, no qual temos as colunas com a média e o desvio padrão dos atrasos.  

Também foi adicionada uma coluna que transforma os códigos presentes no texto nas siglas das UFs.  

Por fim, foi feito um dicionário, "zipando" a coluna UF com um zip entre a média e o desvio padrão.  

Assim, temos um dicionário no qual as chaves são as siglas de UF e os valores são listas, retornando as médias e os desvios padrões em dias. 

In [172]:
def questao_9(datapath = DATA):
    tabela = pd.read_parquet(datapath)
    # Criando as colunas com as datas
    tabela["DT_NOTIFICACAO"] = pd.to_datetime(tabela["DT_NOTIFIC"])
    tabela["DT_SINTOMAS"] = pd.to_datetime(tabela["DT_SIN_PRI"])
    tabela["ATRASO_NOT"] = tabela["DT_NOTIFICACAO"] - tabela["DT_SINTOMAS"]
    # Transformando os números da tabela de datas para inteiros
    tabela["ATRASO_NOT"] = tabela["ATRASO_NOT"].dt.days
    # Criando o dataframe com a média e o desvio padrão
    media_atraso = tabela.groupby("SG_UF_NOT")["ATRASO_NOT"].agg(["mean", "std"]).fillna(0)
    # Colocando o index como número para funcionar
    media_atraso.index = pd.to_numeric(media_atraso.index)
    # Criando uma coluna com as siglas de cada UF
    media_atraso["UF"] = media_atraso.index.map(codigos_estados)
    # Criando o dicionário
    resposta = dict(zip(media_atraso["UF"], zip(media_atraso["mean"].round(2), media_atraso["std"].round(2))))
    return resposta
questao_9()

{'RO': (365.98, 2277.71),
 'RR': (7.0, 7.81),
 'PA': (11.75, 18.44),
 'AP': (191.0, 230.52),
 'TO': (72.25, 101.12),
 'MA': (65.04, 63.38),
 'PI': (40.0, 0.0),
 'CE': (41.57, 64.7),
 'RN': (49.73, 60.82),
 'PB': (135.11, 294.21),
 'PE': (159.64, 498.15),
 'AL': (951.63, 3669.89),
 'SE': (62.6, 323.87),
 'BA': (163.04, 1228.63),
 'MG': (113.22, 829.79),
 'ES': (93.91, 560.26),
 'RJ': (183.59, 233.38),
 'SP': (405.78, 2205.21),
 'PR': (273.92, 1232.03),
 'SC': (22.67, 67.27),
 'RS': (11.0, 12.73),
 'MS': (318.78, 597.05),
 'MT': (615.45, 1085.11),
 'GO': (65.5, 70.96),
 'DF': (254.0, 454.5)}